# Multivariate polynomials implementations

**Contributed by**: Benoît Legat

The SumOfSquares package is built on top of the [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl)
abstract interface. [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
is an implementation of this abstract interface so it can be used with
SumOfSquares. Moreover, any other implementation can be used as well. To
illustrate, we solve Examples 3.38 of [BPT12] with
[TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl),
another implementation of [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl).

[BPT12] Blekherman, G. & Parrilo, P. A. & Thomas, R. R.
*Semidefinite Optimization and Convex Algebraic Geometry*.
Society for Industrial and Applied Mathematics, **2012**.

In [1]:
import TypedPolynomials
TypedPolynomials.@polyvar x y
using SumOfSquares
import CSDP
model = SOSModel(CSDP.Optimizer)
con_ref = @constraint(model, 2x^4 + 5y^4 - x^2*y^2 >= -2(x^3*y + x + 1))
optimize!(model)
solution_summary(model)

Iter: 10 Ap: 9.69e-01 Pobj: -1.0000000e+00 Ad: 9.00e-01 Dobj: -1.0000000e+00 
Success: SDP solved
Primal objective value: -1.0000000e+00 
Dual objective value: -1.0000000e+00 
Relative primal infeasibility: 8.36e-16 
Relative dual infeasibility: 1.91e-09 
Real Relative Gap: -1.02e-09 
XZ Relative Gap: 3.61e-09 
DIMACS error measures: 1.60e-15 0.00e+00 4.55e-09 0.00e+00 -1.02e-09 3.61e-09
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.39e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  9.2771541e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.3401580e+01 


* Solver : CSDP

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution
  Objective value      : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.53172e-01


We see that the problem is feasible. The Sum-of-Squares decomposition can be
obtained as follows:

In [2]:
sos_decomposition(con_ref)

(-0.8025916994817286*x^2 - 0.12696358959695084*x*y + 2.1931536335606596*y^2 - 0.4175974829737731*x - 0.09754582659048604*y - 0.6797576582812246)^2 + (-0.9300185097227984*x^2 - 1.2293230030760685*x*y - 0.10770262293143043*y^2 + 0.2742228250653725*x - 0.6455024395913431*y + 0.9043624572847019)^2 + (0.4443891157327385*x^2 - 0.15182100493580503*x*y - 0.20409230951833635*y^2 - 0.6219741186490518*x - 1.3853650445041525*y - 0.5739128592301723)^2 + (0.020943538499585947*x^2 - 0.7618732513464342*x*y - 0.2555835927433952*y^2 - 0.3436893980484675*x + 0.5184198063947206*y - 0.5702898506776771)^2 + (-0.5053890156943566*x^2 + 0.2972255843408651*x*y - 0.24791041664865918*y^2 - 0.4999809437701394*x + 0.049133727851402666*y + 0.041450896566269956)^2 + (-0.1938395220921123*x^2 + 0.0596091080771317*x*y - 0.10020512085870767*y^2 + 0.2539020228261245*x - 0.06335690502229822*y - 0.25245477072168293)^2

Why is there several implementations ?
Depending in the use-case, one implementation may be more appropriate than
another one. [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is faster than [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
but it requires new compilation whenever the list of variables changes.
This means that [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is not appropriate when the number of variables is dynamic or too large.
However, for a small number of variables, it can be faster.
When solving Sum-of-Squares programs, the time is mostly taken by the Semidefinite programming solver.
The time taken by SumOfSquares/JuMP/MathOptInterface are usually negligible
or it time is taken by manipulation of JuMP or MathOptInterface functions
therefore using TypedPolynomials over DynamicPolynomials may not make much difference in most cases.

One case for which using TypedPolynomials might be adequate is when
using domain defined by equalities (possibly also with inequalities).
Indeed, in that case, SumOfSquares computes the corresponding Gröbner basis which
may take a non-negligible amount of time for large systems of equalities.

To illustrate this, consider the computation of Gröbner basis for the
following system from [CLO05, p. 17].
The time taken by TypedPolynomials is below:

[CLO05] Cox, A. David & Little, John & O'Shea, Donal
*Using Algebraic Geometry*.
Graduate Texts in Mathematics, **2005**.
https://doi.org/10.1007/b138611

In [3]:
using BenchmarkTools
@btime let
    TypedPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.computegröbnerbasis!(S.I)
end

  171.298 μs (3261 allocations: 448.44 KiB)


true

The time taken by DynamicPolynomials is as follows:

In [4]:
import DynamicPolynomials
@btime let
    DynamicPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.computegröbnerbasis!(S.I)
end

  527.694 μs (6381 allocations: 584.00 KiB)


true

We see that TypedPolynomials is faster.
The time is still negligible for this small system but for larger systems, choosing TypedPolynomials may be helpful.
We can use this system in a Sum-of-Squares constraint as follows:

In [5]:
TypedPolynomials.@polyvar x y
S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
poly = -6x - 4y^3 + 2x*y^2 + 6x^3 - 3y^4 + 13x^2 * y^2
model = Model(CSDP.Optimizer)
con_ref = @constraint(model, poly in SOSCone(), domain = S)
optimize!(model)
solution_summary(model)

Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.65e-16 
Relative dual infeasibility: 5.00e-11 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 3.04e-10 
DIMACS error measures: 2.05e-16 0.00e+00 5.00e-11 0.00e+00 0.00e+00 3.04e-10
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.64e-01 Pobj:  0.0000000e+00 Ad: 8.07e-01 Dobj:  2.6104524e+00 
Iter:  2 Ap: 7.90e-01 Pobj:  0.0000000e+00 Ad: 7.40e-01 Dobj: -5.9858991e-02 
Iter:  3 Ap: 7.78e-01 Pobj:  0.0000000e+00 Ad: 7.55e-01 Dobj:  5.6667085e-01 
Iter:  4 Ap: 7.59e-01 Pobj:  0.0000000e+00 Ad: 8.08e-01 Dobj:  4.9238811e-01 
Iter:  5 Ap: 7.15e-01 Pobj:  0.0000000e+00 Ad: 6.42e-01 Dobj:  4.3470711e-01 
Iter:  6 Ap: 6.45e-01 Pobj:  0.0000000e+00 Ad: 7.00e-01 Dobj:  2.1752403e-01 
Iter:  7 Ap: 6.50e-01 Pobj:  0.0000000e+00 Ad: 6.29e-01 Dobj:  1.2996035e-01 
Iter:  8 Ap: 

* Solver : CSDP

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution
  Objective value      : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 4.37679e-01


We obtain the following decomposition:

In [6]:
dec = sos_decomposition(con_ref, 1e-6)

(-24.927154511803916*x^2 + 27.80018416438058*x*y - 6.940895093277724*y^2 - 0.00025745451936590787*x + 0.0004423818427874239*y + 7.5578933517455e-5)^2 + (-21.341702974684363*x^2 - 13.612740134195253*x*y + 22.122686306231827*y^2 - 0.0006202089859727374*x - 0.0004698520241215515*y - 1.6865925877058746e-5)^2 + (-0.07015469214202973*x^2 - 0.09428679976415344*x*y - 0.1256954096544*y^2 + 0.008518353928738955*x - 0.006590600623428728*y + 4.6231622082761956e-5)^2 + (-0.0017527827109975538*x^2 - 0.0023572903711731527*x*y - 0.0031420558911174003*y^2 - 0.054529602188292184*x + 0.04182710745419117*y - 2.3524778609741738e-5)^2

We can verify that it is correct as follows:

In [7]:
rem(dec - poly, S.I)

-7.953798625398231e-6x² - 2.0765851549731616e-5xy - 1.3525048331122e-5y² - 1.8162248153293916e-7x - 2.391663729004229e-7y + 8.687412736367007e-9

Note that the difference between `dec` and `poly` is larger
than between the full gram matrix because `dec` is obtained by dropping
the lowest eigenvalues with the threshold `1e-6`; see `sos_decomposition`.

In [8]:
rem(gram_matrix(con_ref) - poly, S.I)

0.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*